In [6]:
import requests

import random
import matplotlib.pyplot as plt
import math
import argparse
import time
import json
import datetime
import dateutil
from tqdm.notebook import tqdm

nb = 49.27024149430249
sb = 49.25752305948695
wb = -123.25382384052033
eb = -123.24365280903588
x_step = 0.00015
y_step = 0.00015

def randomLocation():
    lat = random.random() * (nb - sb) + sb
    lng = random.random() * (eb - wb) + wb
    return (lat, lng)


In [27]:
class User:
    def __init__(self):
        self.lat, self.lng = randomLocation()
        self.status = 'idle'
        self.task = None
        self.completed = 0
        self.dismissed = 0
        self.avgWalkingSpeed = 0
        self.numWalks = 0
        self.walkingSpeedSum = 0

    def setTask(self, task):
        assert self.task is None
        self.task = task
        self.task.increment()

    def walk(self):
        self.numWalks += 1
        speed = 2.0*random.random()+1.0
        self.walkingSpeedSum += speed
        self.avgWalkingSpeed = self.walkingSpeedSum / self.numWalks
        direction = 0
        if self.status == 'busy':
            dx = self.task.lng - self.lng
            dy = self.task.lat - self.lat
            direction = math.atan2(dy, dx)
        else:
            direction = random.random() * 2 * math.pi
        
        
        dx = math.cos(direction) * speed * x_step
        dy = math.sin(direction) * speed * y_step
        nx = self.lng + dx
        ny = self.lat + dy
        if nx > eb or nx < wb:
            nx -= dx * 2
        if ny > nb or ny < sb:
            ny -= dy * 2
        self.lng = nx
        self.lat = ny

    def decideOnTask(self):
        rand = random.random()
        if rand > 0.3:
            self.status = 'busy'
        else:
            self.finishTask(completed=False)

    def isClose2Task(self):
        if self.task is None:
            return False
        dist = (self.lat - self.task.lat) ** 2 + \
            (self.lng - self.task.lng) ** 2
        return dist <= 4e-6

    def finishTask(self, completed=True):
        self.status = 'idle'
        self.task.decrement()
        self.task = None
        if completed:
            self.completed += 1
        else:
            self.dismissed += 1

    def toRow(self):
        """
        returns user matrices, where matrix = [lat, long, unknown, idle, waiting, busy, avg walking speed, completion ratio]
        """
        arr = [0] * 8
        arr[0] = self.lat
        arr[1] = self.lng
        status = self.status
        if status == 'unknown': arr[2] = 1
        if status == 'idle': arr[3] = 1
        if status == 'waiting': arr[4] = 1
        if status == 'busy': arr[5] = 1
        arr[6] = self.avgWalkingSpeed
        arr[7] = self.completed / (self.completed + self.dismissed)
        return arr

In [28]:
class Task:
    def __init__(self):
        self.lat, self.lng = randomLocation()
        self.targetAoI = 10
        self.numUsers = 0
    
    def decrement(self):
        self.numUsers -= 1

    def increment(self):
        self.numUsers += 1
    
    def toRow(self):
        """
        returns task matrices, matrix = [lat, long, targetAoI, num_users]
        """
        arr = [0] * 4
        arr[0] = self.lat
        arr[1] = self.lng
        arr[2] = self.targetAoI
        arr[3] = self.numUsers
        return arr

    

In [29]:
class Emulator:
    def __init__(self, num_timeslots, num_users, num_tasks):
        super().__init__()
        self.num_timeslots = num_timeslots
        self.users = []
        self.tasks = []
        self.userLocations = [[] for _ in range(num_users)]
        for _ in range(num_users):
            self.users.append(User())
        for _ in range(num_tasks):
            self.tasks.append(Task())

    def getMatrixOf(self, arr):
        matrix = []
        for e in arr:
            matrix.append(e.toRow())
        return matrix

    def assignRandomTask2Users(self):
        for user in self.users:
            if user.status == 'busy':
                continue
            randomTask = random.choice(self.tasks)
            user.setTask(randomTask)
            user.decideOnTask()
    
    def recordUserMoves(self, idx):
        user = self.users[idx]
        location = (user.lat, user.lng)
        self.userLocations[idx].append(location)
    
    def run(self):
        for cycle in tqdm(range(self.num_timeslots)):
            self.assignRandomTask2Users()
            for i in range(len(self.users)):
                user = self.users[i]
                user.walk()
                self.recordUserMoves(i)
                if user.isClose2Task():
                    user.finishTask()


In [30]:
emulator = Emulator(num_timeslots=100, num_tasks=5, num_users=3)
emulator.run()


In [31]:
emulator.getMatrixOf(emulator.tasks)

[[49.27018093894551, -123.2501070302122, 10, 1],
 [49.26107341435841, -123.25311234359505, 10, 0],
 [49.26584172243786, -123.24857693150405, 10, 1],
 [49.25894551633051, -123.24468930604647, 10, 0],
 [49.25972129730647, -123.24520043272604, 10, 1]]